In [1]:
from datasets import load_dataset

dataset = load_dataset('wikitext', 'wikitext-103-v1')

- Concatenate every title with the following paragraphs until the next title
- One training example: `BOS + TITLE + PARAGRAPHS + EOS`

In [2]:
train_text = dataset['train']['text']
train_text[1]

' = Valkyria Chronicles III = \n'

In [206]:
def is_title(text):
    has_format = (text[:3] == ' = ' and text[-4:] == ' = \n')
    return has_format and text[3].isupper()

is_title(train_text[1])

True

In [207]:
examples = []
example = train_text[1]  # train_text[0] is emtpy

for text in train_text[2:]:
    if is_title(text):
        examples.append(example)
        example = text
    else:
        example += text

In [208]:
len(examples)

27571

In [ ]:
ex_idxs = sorted(range(len(examples)), key=lambda i: len(examples[i]))

In [232]:
i = 0
print(*examples[ex_idxs[i]-1:ex_idxs[i]+2])

 = Rank in the league ; P = 
 Played ; W 
  = Win ; D = 
 Draw ; L 
  = Loss ; F = 
 Goals for ; A 



In [233]:
import mlx.core as mx
from tqdm.notebook import tqdm
from sentencepiece import SentencePieceProcessor

sp_model = SentencePieceProcessor(model_file='tokenizer.model')

def encode(text):
    tokens = sp_model.encode(text, add_bos=True, add_eos=True)
    token_arr = mx.array(tokens, dtype=mx.uint16)
    return token_arr

train_tokens = [encode(example) for example in tqdm(examples)]

  0%|          | 0/27571 [00:00<?, ?it/s]

In [234]:
len(train_tokens)

27571

In [235]:
print(sp_model.decode(train_tokens[0].tolist()))

 = Valkyria Chronicles III = 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . 

In [236]:
chunk_size = 1000
for chunk_idx, idx in enumerate(range(0, len(train_tokens), chunk_size)):
    mx.savez(f'wikitext_data/train/example_chunk{chunk_idx:02d}', *train_tokens[idx:idx+chunk_size])

In [237]:
from pathlib import Path
import mlx.core as mx

train_data_dir = Path('./wikitext_data/train')
train_examples = []

for ex_path in sorted(train_data_dir.glob('*.npz')):
    train_examples.extend(mx.load(str(ex_path)).values())

len(train_examples)

27571

In [238]:
sp_model.decode(train_examples[0].tolist())

' = Lloyd Mathews = \n Sir Lloyd William Mathews , GCMG , CB ( 7 March 1850 – 11 October 1901 ) was a British naval officer , politician and abolitionist . Mathews joined the Royal Navy as a cadet at the age of 13 and progressed through the ranks to lieutenant . He was involved with the Third Anglo @-@ Ashanti War of 1873 – 4 , afterwards being stationed in East Africa for the suppression of the slave trade . In 1877 he was seconded from the navy to Sultan Barghash of Zanzibar in order to form a European @-@ style army ; he would remain in the employment of the government of Zanzibar for the rest of his life . His army quickly reached 6 @,@ 300 men and was used in several expeditions to suppress the slave trade and rebellions against the Zanzibar government . \n Mathews retired from the Royal Navy in 1881 and was appointed Brigadier @-@ General of Zanzibar . There followed more expeditions to the African mainland , including a failed attempt to stop German expansion in East Africa . In